## GUI init
After initializing a session object, we initialize the GUI :

    # GUI Initialization
    root = tk.Tk()  
    myApp = tk_gui.Interface(root, my_sessions[working_session], "PaPL")  # Create a window based on root 
    root.protocol("WM_DELETE_WINDOW", myApp.on_delete)  # Function to call on session closing
    
As the GUI updates functions take in input some session element, we will present it later

In [1]:
# Ttkinter for the GUI and PIL for importing image from OpenCV to tk
import tkinter as tk
from tkinter import ttk
from PIL import Image

# To create a session object
import sys, os
sys.path.insert(1, './src')
import session
import save_load

# GUI, Tkinter
Now that we have present all the image processing part, we can go through the GUI which is made with Tkinter. You can find some good explanation and examples of tkinter in the following websites :
    - https://docs.python.org/
    - https://www.tutorialspoint.com/python/python_gui_programming.htm
    - https://kite.com/python/docs/ttk.Style
    - https://effbot.org/tkinterbook/
    - https://www.tcl.tk/man/tcl8.6/contents.htm
    - https://tkdocs.com/

There's 3 classes in the tk_gui.py file:
    - PopupWindowGet to get information in a popup window
    - PopupWindowInfo to show information in a popup window
    - Interface which is our main window and contains all the information to be shown to the user
    
We are presenting the Popup classes first because they are simpler and very similar.

In [2]:
# Constants definition

WIDTH = 1200
H_OFFSET = 100
HEIGHT = 675
V_OFFSET = 100
SIZE = "{0}x{1}+{2}+{3}".format(WIDTH, HEIGHT, H_OFFSET, V_OFFSET)  # width x height + horizontal offset + vertical
# offset
PADDING = 5
MARGIN = 200

### PopupWindowGet

It has an attribute value which is the one returned when closing the popup. 
Every thing else is tkinter library and will be explained step by step. 

In [3]:
class PopupWindowGet:
    """Get value information popup"""

    def __init__(self, master, title, text, choices=None):
        self.value = None  # Value that will be set when closing the popup
        
        # Define a new style for popup buttons
        self.button_style = ttk.Style()
        self.button_style.configure('Popup.TButton',  # Name of the style
                                    font=('calibri', 20, 'bold'),
                                    borderwidth='4')
        # When the mouse is on the button
        self.button_style.map('Popup.TButton',
                              foreground=[('active', '!disabled', 'green')],
                              background=[('active', 'black')])
        try:
            top = self.top = tk.Toplevel(master)
        except tk.TclError:  # If the master window has been closed
            return
        self.top.title(title)
        self.input_text = ttk.Label(top, text=text, style='TLabel')
        self.input_text.pack()
        if choices:
            # Next line was used to work without ttk
            # self.output_value = ttk.Combobox(master=top, values=choices)
            
            # Define a Tkinter string variable that can be accessed using the .get() command
            self.output_value = tk.StringVar()
            # Add a button for each choice in choices
            for choice in choices:
                self.button = ttk.Radiobutton(master=top,
                                              text=choice,
                                              variable=self.output_value,
                                              command=self.cleanup,
                                              style='Popup.TButton',
                                              value=choice)
                self.button.pack()
        else:
            # Add an empty label that the user will fill
            self.output_value = tk.Entry(top)
            self.output_value.pack()
            # To be able to write in the label without clicking on it
            self.output_value.focus_set()
            # Finalization button
            self.validation_button = tk.Button(top, text='Ok', command=self.cleanup)
            self.validation_button.pack()
            # To position the popup window with MARGIN from the top left corner 
            self.top.geometry('+{0}+{1}'.format(MARGIN, MARGIN))
            # Binding the Enter key to cleanup function
            top.bind('<Return>', self.cleanup)

    def cleanup(self, event=None):
        self.value = self.output_value.get()
        self.top.destroy()

In [4]:
root = tk.Tk()
root.title('My GUI')
root.update()
root.popup = PopupWindowGet(root, 'Number', 'Please enter a number')
root.wait_window(root.popup.top)
print(str(root.popup.value))
root.popup = PopupWindowGet(root, 'Choice', 'You have a choice to make', [42, 15, 666])
root.wait_window(root.popup.top)
print(str(root.popup.value))
root.destroy()

42
42


### PopupWindowShow

It's exacly the same as PopupWindowGet minus the output_value.

In [5]:
class PopupWindowInfo:
    """Information popup window"""

    def __init__(self, master, title, text):
        try:
            top = self.top = tk.Toplevel(master)
        except tk.TclError:  # If the master window has been closed
            return
        self.top.title(title)
        self.input_text = ttk.Label(top, text=text, style='TLabel')
        self.input_text.pack()
        # We don't need to save any variable when closing
        self.validation_button = ttk.Button(top, text='Ok', command=self.top.destroy, style='Popup.TButton')
        self.validation_button.pack()
        self.top.geometry('+{0}+{1}'.format(MARGIN, MARGIN))

In [6]:
root = tk.Tk()
root.title('My GUI')
root.update()
root.popup = PopupWindowInfo(root, 'Info', 'I just wanted to tell you this')
root.wait_window(root.popup.top)
root.destroy()
print('Finished')

Finished


### Interface

This is our main window. It is a bit more complete than the 2 others because we need to position carefully each label but it follows the same syntax.

It has the following attribute :
    - root which is a link to the root tk.Tk()
    - session, the current session
    - popup, for all temporary popups

The window is by default in fullscreen mode but can be switched with F11 or exit with ESC. Also q and a keys call the same functions as the button.

Label initialization is done with the grid function. It takes as argument row and column, the position of the label, rowspan and columnspan, the width/height of the label (if we need to fuse some cell for a label) and padx, pady which represent the padding. We don't use any other argument here.

The update_gui function, which is the most important, resize every image so they have the correct size so as not to overlap with other label.

In [7]:
class Interface:
    """Main window of the GUI. Shows the live webcam feed, the interpreted program and the last event tile"""

    def __init__(self, root, session, title="Interface"):
        # initialization
        self.root = root
        self.session = session
        self.root.title(title)
        self.popup = None  # Initialization of the popup window

        # geometry initialization
        self.full_screen = True
        self.geometry = SIZE
        self.root.geometry(SIZE)

        # Set the default layout to fullscreen
        self.root.attributes("-fullscreen", False)

        # Set the key bindings
        self.root.bind('<Escape>', self.end_fullscreen)
        self.root.bind('<F11>', self.toggle_fullscreen)
        self.root.bind('q', self.reset_program_fun)
        self.root.bind('a', self.load_program_fun)

        # Style initialization
        self.button_style = ttk.Style()
        self.button_style.configure('Interface.TButton',
                                    font=('calibri', 20),
                                    borderwidth='4')
        # When the button is pressed
        self.button_style.map('Interface.TButton',
                              foreground=[('pressed', 'red')],
                              background=[('pressed', 'black')])
        self.label_style = ttk.Style()
        self.label_style.configure('TLabel',
                                   font=('calibri', 20),
                                   borderwidth='4')

        # labels initialization
        self.webcam_title = ttk.Label(text="Current image from the webcam", style='TLabel')
        self.webcam_title.grid(row=0, column=0, padx=PADDING, pady=PADDING)
        self.webcam = tk.Label()
        self.webcam.grid(row=1, column=0, padx=PADDING, pady=PADDING)
        self.last_event_title = ttk.Label(text="Last event detected by the Thymio", style='TLabel')
        self.last_event_title.grid(row=2, column=0, padx=PADDING, pady=PADDING)
        self.last_event_tile = tk.Label()
        self.last_event_tile.grid(row=3, column=0, rowspan=2, padx=PADDING, pady=PADDING)
        self.logo_label = tk.Label()
        self.logo_label.grid(row=0, column=1, padx=PADDING, pady=PADDING)
        self.program_title = ttk.Label(text="Current program", style='TLabel')
        self.program_title.grid(row=0, column=2, columnspan=2, padx=PADDING, pady=PADDING)
        tk.Grid.columnconfigure(self.root, 2, weight=1)  # To make label of the column 2 the largest as possible
        self.program = tk.Label()
        self.program.grid(row=1, column=1, rowspan=3, columnspan=3, padx=PADDING, pady=PADDING)
        tk.Grid.rowconfigure(self.root, 3, weight=1)  # To make label of the row 2 the largest as possible
        self.load_program = ttk.Button(root, text="load program", command=self.load_program_fun,
                                       style='Interface.TButton')
        self.load_program.grid(row=4, column=1, padx=PADDING, pady=PADDING)
        self.reset_game = ttk.Button(root, text="Game selection", command=self.reset_session_state,
                                     style='Interface.TButton')
        self.reset_game.grid(row=4, column=2, padx=PADDING, pady=PADDING)
        self.reset_program = ttk.Button(root, text="reset program", command=self.reset_program_fun,
                                        style='Interface.TButton')
        self.reset_program.grid(row=4, column=3, padx=PADDING, pady=PADDING)

        # Update to show the labels
        self.update()

    def load_program_fun(self, event=None):
        if self.session.state == session.States.WEBCAM_MAIN:
            self.session.state = session.States.LAUNCH_WEBCAM
        elif self.session.state == session.States.IMG_FILE_MAIN:
            self.session.state = session.States.LAUNCH_FILE
        elif self.session.state == session.States.CAPTURE_MAIN:
            self.session.state = session.States.LAUNCH_CAPTURE

    def reset_program_fun(self, event=None):
        if self.session.state == session.States.LAUNCH_WEBCAM or self.session.state == session.States.EXECUTE_WEBCAM:
            self.session.state = session.States.WEBCAM_MAIN
        elif self.session.state == session.States.LAUNCH_FILE or self.session.state == session.States.EXECUTE_FILE:
            self.session.state = session.States.IMG_FILE_MAIN
        elif self.session.state == session.States.LAUNCH_CAPTURE or self.session.state == session.States.EXECUTE_CAPTURE:
            self.session.state = session.States.CAPTURE_MAIN

    def reset_session_state(self):
        self.session.state = session.States.GAME_CHOICE

    def toggle_fullscreen(self, event=None):
        self.program.configure(image='')
        self.full_screen = not self.full_screen
        self.root.attributes("-fullscreen", self.full_screen)

    def end_fullscreen(self, event=None):
        self.program.configure(image='')
        self.full_screen = False
        self.root.attributes("-fullscreen", False)

    def on_delete(self):
        self.session.state = session.States.EXIT
        # self.popup.destroy()  # Not needed, already handled by self.root.destroy()
        self.root.destroy()

    def update(self):
        self.root.update()
        # self.program.configure(height=int(self.root.winfo_height() - self.program_title.winfo_height()
        #                                   - self.load_program.winfo_height()) - 8 * PADDING)

    def update_gui(self, webcam, program):
        last_event = self.session.current_event_tile
        
        # Add the logo on the top of the main window
        # Another way to resize while keeping aspect ratio. See PIL Documentation for more infos
        size = self.root.winfo_width() - self.webcam_title.winfo_width(), self.program_title.winfo_height()
        logo_copy = self.session.module_logo.copy()  # Make a copy before doing modification
        logo_copy.thumbnail(size, Image.ANTIALIAS)
        logo_tk = ImageTk.PhotoImage(image=logo_copy)
        self.logo_label.configure(image=logo_tk)
        
        # Update the webcam while keeping aspect ratio
        try:
            ratio = webcam.shape[1] / webcam.shape[0]
            webcam_height = int((self.root.winfo_height() - self.webcam_title.winfo_height()
                                 - self.last_event_title.winfo_height()) / 2)
            webcam_width = int(ratio * webcam_height)
            webcam_resized = cv2.resize(webcam, (webcam_width - 6 * PADDING, webcam_height - 6 * PADDING))
            # Convert cv2 image into ImageTk, using PIL library
            b_webcam, g_webcam, r_webcam = cv2.split(webcam_resized)
            img_webcam = cv2.merge((r_webcam, g_webcam, b_webcam))
            img_webcam_pil = Image.fromarray(img_webcam)
            img_webcam_tk = ImageTk.PhotoImage(image=img_webcam_pil)
            self.webcam.configure(image=img_webcam_tk)
        except Exception:
            pass  # Shouldn't happen, except when closing the application
        
        # Update the webcam while keeping aspect ratio
        try:
            ratio = last_event.shape[1] / last_event.shape[0]
            last_event_height = webcam_height
            last_event_width = int(ratio * last_event_height)
            last_event_resized = cv2.resize(last_event,
                                            (last_event_width - 6 * PADDING, last_event_height - 6 * PADDING))
            # Convert cv2 image into ImageTk, using PIL library
            b_last_event, g_last_event, r_last_event = cv2.split(last_event_resized)
            img_last_event = cv2.merge((r_last_event, g_last_event, b_last_event))
            img_last_event_pil = Image.fromarray(img_last_event)
            img_last_event_tk = ImageTk.PhotoImage(image=img_last_event_pil)
            self.last_event_tile.configure(image=img_last_event_tk)
        except Exception:
            pass  # Keep the last event as long as there is no new event
        
        # Update the program while keeping aspect ratio
        try:
            ratio = program.shape[1] / program.shape[0]
            program_height = int(self.root.winfo_height() - self.program_title.winfo_height()
                                 - self.load_program.winfo_height())
            program_width = int(self.root.winfo_width() - webcam_width)
            if ratio * program_height >= program_width:
                program_height = int(program_width / ratio)
            else:
                program_width = int(program_height * ratio)
            program_resized = cv2.resize(program, (program_width - 8 * PADDING, program_height - 8 * PADDING))
            # Convert cv2 image into ImageTk, using PIL library
            b_program, g_program, r_program = cv2.split(program_resized)
            img_program = cv2.merge((r_program, g_program, b_program))
            img_program_pil = Image.fromarray(img_program)
            img_program_tk = ImageTk.PhotoImage(image=img_program_pil)
            self.program.configure(image=img_program_tk)
        except Exception:
            self.program.configure(image='')  # Convention of TKinter. Delete the last program if new is empty.
        try:
            tk.Grid.columnconfigure(self.root, 2, weight=1)  # To make label of the column 2 the largest as possible
            tk.Grid.rowconfigure(self.root, 3, weight=1)  # To make label of the row 2 the largest as possible
            self.root.update()
        except tk.TclError:  # if application has been destroyed
            pass

    def get_integer(self, title, text):
        self.popup = PopupWindowGet(self.root, title, text)
        try:
            self.root.wait_window(self.popup.top)
            return int(self.popup.value)
        except ValueError:  # If value isn't an Integer
            return
        except AttributeError:  # If window has been closed
            return
        except TypeError:  # If program has been closed
            return

    def get_choice(self, title, text, choices):
        self.popup = PopupWindowGet(self.root, title, text, choices)
        try:
            self.root.wait_window(self.popup.top)
            return str(self.popup.value)
        except ValueError:  # If value isn't a string
            return
        except AttributeError:  # If window has been closed
            return
        except TypeError:  # If program has been closed
            return

    def get_text(self, title, text):
        self.popup = PopupWindowGet(self.root, title, text)
        try:
            self.root.wait_window(self.popup.top)
            return str(self.popup.value)
        except ValueError:  # If value isn't a string
            return
        except AttributeError:  # If window has been closed
            return
        except TypeError:  # If program has been closed
            return

    def show_info(self, title, text):
        self.popup = PopupWindowInfo(self.root, title, text)
        self.root.wait_window(self.popup.top)

We need to create a session in order to use our GUI.

In [8]:
my_session = session.Session()
my_session.module = session.Module.THYMIO
my_session.module_logo = Image.open(os.getcwd() + "/data/logo/thymio_logo.png")
tile_library, tile_library_resized, tile_type = save_load.load_data(session.Module.THYMIO)

my_session.tile_library = tile_library
my_session.tile_library_resized = tile_library_resized
my_session.tile_type = tile_type
my_session.state = session.States.IMG_SOURCE_CHOICE  # We have automatically chosen the Thymio module here

Now we can create our interface. Unfortunately, Tkinter doesn't work well in a Jupyter Notebook so you probably won't be able too click on button and call function through keys binding.

F11 is to switch fullscreen and Escape to exit fullscreen mode. Button changes the state of session but it is not visible in this example (it has impact only when the state machine is handled in main).

In [ ]:
# GUI Initialization
root = tk.Tk()
myApp = Interface(root, my_session, "PaPL")
root.protocol("WM_DELETE_WINDOW", myApp.on_delete)
while True:
    try:
        root.update()
    except tk.TclError:
        pass